# Semantic Segmentation - Metrics & Losses

In [3]:
import numpy as np
import tensorflow as tf

2022-10-13 18:59:35.942726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu::/usr/lib/hadoop/lib/native
2022-10-13 18:59:35.942870: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-13 18:59:36.072590: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-13 18:59:40.138531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu::/usr/lib/hadoop/lib/native
2022-10-13 18:59:40.140344: W tensorflow

## Dummy Data

Let's build dummy examples of a `y_true` ground truth mask and a `y_pred` prediction mask to help understand the inner workings of the various loss functions.

#### Ground Truth Mask

In [4]:
# ground truth mask
# set first channel as background, 2 pixel defect
y_true = np.zeros((1, 4, 4, 3))
y_true[0, :, :, 0] = 1.0
y_true[0, (1,2), 1, 0] = 0.0
y_true[0, (1,2), 1, 1] = 1.0

In [5]:
y_true[0, :, :, 0], y_true[0, :, :, 1], y_true[0, :, :, 2]

(array([[1., 1., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 1., 1., 1.]]),
 array([[0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))

#### Prediction Mask

In [6]:
# pred_mask
# set first channel as background, 2 pixel defect
y_pred = np.zeros((1, 4, 4, 3))
y_pred[0, :, :, 0] = 1.0

In [7]:
y_pred[0, :, :, 0], y_pred[0, :, :, 1], y_pred[0, :, :, 2]

(array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))

## Jaccard Index | Intersection over Union (IoU)

The Jaccard Index (aka Intersection-Over-Union) is a common evaluation metric for semantic segmentation.

<center><img src="../images/IoU.png"/></center>


$$
...
$$

$$ IoU = \frac{X \cap Y }{X \cup Y } = \frac{TP}{TP + FP + FN} $$

$$
...
$$

When applied to boolean data, we can represent IoU with true postitives (TP), false positives (FP), and false negatives (FN), as seen above.

[Image Credit](https://en.wikipedia.org/wiki/Jaccard_index)

In [8]:
def iou(y_true, y_pred, smooth=1e-6):

    y_true_pos = tf.keras.layers.Flatten()(y_true)
    y_pred_pos = tf.keras.layers.Flatten()(y_pred)

    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos))
    false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos)


    return (true_pos + smooth) / (
        true_pos + false_pos + false_neg + smooth
    )

In [9]:
iou(y_true, y_pred).numpy()

2022-10-13 18:59:48.989573: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu::/usr/lib/hadoop/lib/native
2022-10-13 18:59:48.989760: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-13 18:59:48.989877: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (uyj6gicxuni7nri3): /proc/driver/nvidia/version does not exist


0.77777773

## Dice Similarity Coefficient & Loss

The dice similarity coefficient (DSC) is a commonly used measure of overlap between a predicted and ground truth mask. The dice coefficient excludes the background class from the loss calculation so that error signal is attributed to defect mask predictions only. This ensures the pixel-wise class imbalance of the background class does not dominate loss contributions in comparison to the underrepresented defect pixels.

The dice coefficient ranges from 0-1, where 1 represents absolute overlap, while 0 indicates no overlap at all. Therefore, we can use 1 - DSC as a representative loss function that we aim to minimize.

<center><img src="../images/dice_coeff.png"/></center>

$$
...
$$

$$ DSC = \frac{2 | X \cap Y | }{|X| + |Y|} = \frac{2TP}{2TP + FP + FN} $$

$$
...
$$

Here, $|X|$ and $|Y|$ represent the number of elements in each set. Therefore, DSC is twice the number of pixels in common between two masks divided by the sum of number of pixels in each mask. When applied to boolean data, we can represent DSC with true postitives (TP), false positives (FP), and false negatives (FN), as seen above.

[Image Credit](https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.kaggle.com%2Fcode%2Fyerramvarun%2Funderstanding-dice-coefficient&psig=AOvVaw2kmPKQbhzdAWBKpJ0xzmtw&ust=1665757231787000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCICjp7Oz3foCFQAAAAAdAAAAABAM_)

In [56]:
def dice_coeff(y_true, y_pred, smooth=1e-6, remove_bkg=False):

    if remove_bkg:
        # remove background channel from loss calculation
        y_true = y_true[:, :, :, 1:]
        y_pred = y_pred[:, :, :, 1:]

    y_true_pos = tf.keras.layers.Flatten()(y_true)
    y_pred_pos = tf.keras.layers.Flatten()(y_pred)

    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos))
    false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos)


    return (2.0 * true_pos + smooth) / (
        2.0 *true_pos + false_pos + false_neg + smooth
    )

def dice_loss(y_true, y_pred, smooth=1e-6, remove_bkg=False):
    loss = 1 - dice_coeff(y_true, y_pred, smooth, remove_bkg)
    return loss


In [57]:
dice_loss(y_true, y_pred, remove_bkg=False).numpy(), dice_loss(y_true, y_pred, remove_bkg=True).numpy()

(0.12499994, 0.9999995)

Here we see that when the background class is included in the loss calculation, loss is relatively low (0.12) despite the fact that the prediction mask didn't correctly predict a single defect pixel. By removing the background class from the calculation, the loss signal focuses solely on defective classes. As we see, this results in a high loss (0.99) as we would expect.

## Tversky Index

As seen above, Dice Loss is a harmonic mean of precision and recall that weights false positives and false negatives equally. The Tversky Index is a generalization of Dice Similarity Coefficient that affords a tunable parameter to weight FP's and FN's differently. This tradeoff between precision and recall allows us to place more emphasis on false negative, which are commonly the issue when dealing with highly imbalanced data. 

$$
...
$$

$$ TI = \frac{TP}{TP + \alpha FP + (1-\alpha)FN} $$

$$
...
$$

Here, Tversky Index introduces the $\alpha$ parameter. In the case where $\alpha=0.5$, TI simplifies to the dice coefficient. However by setting  $\alpha>0.5$ we can enforce a higher penalty on false negatives.


In [65]:
def tversky(y_true, y_pred, alpha = 0.7, smooth=1e-6):
    # Focal Tversky loss, brought to you by:  https://github.com/nabsabraham/focal-tversky-unet

    # remove background channel from loss calculation
    y_true = y_true[:, :, :, 1:]
    y_pred = y_pred[:, :, :, 1:]

    y_true_pos = tf.keras.layers.Flatten()(y_true)
    y_pred_pos = tf.keras.layers.Flatten()(y_pred)
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos))
    false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos)


    return (true_pos + smooth) / (
        true_pos + (alpha * false_neg) + ((1 - alpha) * false_pos) + smooth
    )

def tversky_loss(y_true, y_pred, alpha = 0.7, smooth=1e-6):
    return 1 - tversky(y_true, y_pred, alpha, smooth)

In [67]:
tversky_loss(y_true, y_pred, alpha=0.1).numpy(), tversky_loss(y_true, y_pred, alpha=0.9).numpy()

(0.999995, 0.99999946)

## References

- [Tversky loss function for image segmentation using 3D fully convolutional deep networks](https://arxiv.org/abs/1706.05721)
- [A Novel Focal Tversky Loss Function with Improved Attention U-Net for Lesion Segmentation](https://arxiv.org/pdf/1810.07842.pdf)
- [Dealing with class imbalanced image datasets using Focal Tversky Loss](https://towardsdatascience.com/dealing-with-class-imbalanced-image-datasets-1cbd17de76b5)

## Per class evaluation

In [10]:
y_true[0, :, :, 0], y_true[0, :, :, 1], y_true[0, :, :, 2]

(array([[1., 1., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 1., 1., 1.]]),
 array([[0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))

In [11]:
y_pred[0, :, :, 0], y_pred[0, :, :, 1], y_pred[0, :, :, 2]

(array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))

In [25]:
def dice_coeff_per_class(y_true, y_pred, smooth=1e-6, remove_bkg=False):

    metrics = {}

    for class_idx in range(y_true.shape[-1]):

        y_true_class = y_true[..., class_idx]
        y_pred_class = y_pred[..., class_idx]

        y_true_pos = tf.keras.layers.Flatten()(y_true_class)
        y_pred_pos = tf.keras.layers.Flatten()(y_pred_class)

        true_pos = tf.reduce_sum(y_true_pos * y_pred_pos)
        false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos))
        false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos)

        score = (2.0 * true_pos + smooth) / (
        2.0 *true_pos + false_pos + false_neg + smooth
    )

        metrics[class_idx] = score.numpy()

    return metrics

In [26]:
dice_coeff_per_class(y_true, y_pred)

{0: 0.93333334, 1: 4.9999977e-07, 2: 1.0}